## EDA And Feature Engineering Flight Price Prediction
check the dataset info below
https://www.kaggle.com/datasets/shubhambathwal/flight-price-prediction

### FEATURES
The various features of the cleaned dataset are explained below:
1) Airline: The name of the airline company is stored in the airline column. It is a categorical feature having 6 different airlines.
2) Flight: Flight stores information regarding the plane's flight code. It is a categorical feature.
3) Source City: City from which the flight takes off. It is a categorical feature having 6 unique cities.
4) Departure Time: This is a derived categorical feature obtained created by grouping time periods into bins. It stores information about the departure time and have 6 unique time labels.
5) Stops: A categorical feature with 3 distinct values that stores the number of stops between the source and destination cities.
6) Arrival Time: This is a derived categorical feature created by grouping time intervals into bins. It has six distinct time labels and keeps information about the arrival time.
7) Destination City: City where the flight will land. It is a categorical feature having 6 unique cities.
8) Class: A categorical feature that contains information on seat class; it has two distinct values: Business and Economy.
9) Duration: A continuous feature that displays the overall amount of time it takes to travel between cities in hours.
10)Days Left: This is a derived characteristic that is calculated by subtracting the trip date by the booking date.
11) Price: Target variable stores information of the ticket price.

### Summary of how to clean data
1) **Data Cleaning**
- Replace nulls or nan values with 
   - replace numeric data with mean or median or mode - fillna()
   - drop columns or rows with nulls if it applies - dropna()
- Delete input columns which do not have a direct effect in determining the output columns

2) **EDA (Exploratory Data Analysis)**
- Understand the structure of data
   - df.head(), df.tail()
   - df.shape
   - df.columns
   - df.info
   - df.describe()
   - df.shape()
   - Count and sum of nulls and missing values
   - Check for outliers - 5 point summary and Box Plot, Scatter plots, Distribution plots
   - Co-relation analysis - df.corr(), heatmap, Spearman collection (non-linear), Kendall co-relation

3) **Feature Engineering**
- Apply Encoding techniques - Encode Category based data using the relevant Encoding technique e.g. OHE (One Hot Encoding)


### Goal
- The Goal is to take into account all the input features(date of journey, source, destination, etc) and predict the output feature (price) for an airline.


In [5]:
#importing basics libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# this is a Juypter 'magic' command
# It tells Jupyter to display Matplotlib plots directly inside the notebook, right below the cell that creates them.
# It also makes plots appear automatically without needing plt.show() (in most simple cases).
# It only works in Jupyter/IPython environments; it has no effect in a normal Python script.
%matplotlib inline

In [61]:
# ************* Load the airline data and get Statistics on existing data **********
df=pd.read_excel('./resources/flight_price.xlsx')
df.head()
df.tail()

## Get the basics info about data
df.info()

# (IMPORTANT) - df.describe() only gives the statistical(mean, median, Q1, Q3, etc) values of numerical columns 
# At this time we have only one numerical column - Price
# To get stats from other columns we have to TYPECAST them to int.
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [ ]:
## ******** EDA > Cleaning data *****************
# Optimizing Date of Journey column
# 1) Splitting Date of Journey into Date, Month and Year
# 2) Converting Date, Month and Year into (int) so ML can deal with it in a better way.
# 3) Drop the 'Date of Journey' column

# 1) xtract three additional columns Date, Month and Year from 'Date of Journey'.
# Date, Month and Year are of Object type. 
df['Date']=df['Date_of_Journey'].str.split('/').str[0]
df['Month']=df['Date_of_Journey'].str.split('/').str[1]
df['Year']=df['Date_of_Journey'].str.split('/').str[2]

df.info()
df.head()

#2)  Convert date, month and year into integers, so we can run some statistical analysis on them
df['Date']=df['Date'].astype(int)
df['Month']=df['Month'].astype(int)
df['Year']=df['Year'].astype(int)
df.info();
df.describe();

# Date, Month and Year are now listed as seperate columns
# Date of Journey (the origional column from where Date, Month and Year are split) are also listed.
df.head()

# 3) Drop Date Of Journey since it is not coverted to three columns - Date, Month and Year
df.drop('Date_of_Journey',axis=1,inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
 11  Date             10683 non-null  object
 12  Month            10683 non-null  object
 13  Year             10683 non-null  object
dtypes: int64(1), object(13)
memory usage: 1.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 14 columns):
 #   Column

In [10]:
# Optimizing 'Arrival time' column
# 1) Splitting 'Arrival Time' before and after space. 01:10 22 Mar will become 01:10, we only need the time, not the date
# 2) Split 'Arrival Time' into 'Arrival hour' and 'Arrival min'
# 3) Convert 'Arrival hour' and 'Arrival min' to int

# 1) Splitting 'Arrival Time' before and after space. 01:10 22 Mar will become 01:10, we only need the time, not the date
df['Arrival_Time']=df['Arrival_Time'].apply(lambda x:x.split(' ')[0])
print('arrival time = ',df['Arrival_Time'])

# 2) Split 'Arrival Time' into 'Arrival hour' and 'Arrival min'
df['Arrival_hour']=df['Arrival_Time'].str.split(':').str[0]
df['Arrival_min']=df['Arrival_Time'].str.split(':').str[1]

#3) Convert 'Arrival hour' and 'Arrival min' to int
df['Arrival_hour']=df['Arrival_hour'].astype(int)
df['Arrival_min']=df['Arrival_min'].astype(int)

#4) Drop the origional column 'Arrival Time' (from which 'Arrival Hour' and 'Arrival Min' were split)
df.drop('Arrival_Time',axis=1,inplace=True)

df.head(2)

arrival time =  0        01:10
1        13:15
2        04:25
3        23:30
4        21:35
         ...  
10678    22:25
10679    23:20
10680    11:20
10681    14:10
10682    19:15
Name: Arrival_Time, Length: 10683, dtype: object


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Duration,Total_Stops,Additional_Info,Price,Arrival_hour,Arrival_min
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,2h 50m,non-stop,No info,3897,1,10
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,7h 25m,2 stops,No info,7662,13,15


In [36]:
# Optimizing 'Dep_time' column
#1) Split 'Dep_time' to 'Departure_hour' and 'Departure_min'
#2) Convert 'Departure_hour' and 'Departure_min' to (int)
#3) Drop the column 'Dep_time'

#1) Split 'Dep_time' to 'Departure_hour' and 'Departure_min'
df['Departure_hour']=df['Dep_Time'].str.split(':').str[0]
df['Departure_min']=df['Dep_Time'].str.split(':').str[1]

#2) Convert 'Departure_hour' and 'Departure_min' to (int)
df['Departure_hour']=df['Departure_hour'].astype(int)
df['Departure_min']=df['Departure_min'].astype(int)

#3) Drop the column 'Dep_time'
df.drop('Dep_Time',axis=1,inplace=True)

# 'Price', 'Arrival hour','Arrival min','Departure hour','Departure min' are all (int) datatypes now.
df.info()
df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Airline          10683 non-null  object 
 1   Date_of_Journey  10683 non-null  object 
 2   Source           10683 non-null  object 
 3   Destination      10683 non-null  object 
 4   Route            10682 non-null  object 
 5   Arrival_Time     10683 non-null  object 
 6   Duration         10683 non-null  object 
 7   Total_Stops      0 non-null      float64
 8   Additional_Info  10683 non-null  object 
 9   Price            10683 non-null  int64  
 10  Total_Stops2     10683 non-null  int64  
 11  Departure_hour   10683 non-null  int64  
 12  Departure_min    10683 non-null  int64  
dtypes: float64(1), int64(4), object(8)
memory usage: 1.1+ MB


,Airline,Date_of_Journey,Source,Destination,Route,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Total_Stops2,Departure_hour,Departure_min
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,01:10 22 Mar,2h 50m,NaN,No info,3897,1,22,20
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,13:15,7h 25m,NaN,No info,7662,1,5,50


In [45]:
# Optimizing 'Total stops' column  with values like (non-stop, 2 stops, 3 stops)
# This is a categorical column
#1) Print all unique values to get all the category types in this column
#2) Find any rows where Total Stops is null

#1) Print all unique values to get all the category types in this column
print('Unique values in `Total Stops` ',df['Total_Stops'].unique())

#2) Find any rows where Total Stops is null - only ONE row
print('Total stops is null in the following rows = ',df[df['Total_Stops'].isnull()]);

#3) Find the mode (most values present) 
# Output: Two modes = 0, 1 stop
print('Mode of total stops = ',df['Total_Stops'].mode());

#4) Convert 'Total stops' to a numeric value where : 0=non-stop, 1=stop, 2=2 stops, etc
df['Total_Stops_Optimized']=df['Total_Stops'].map({'non-stop':0,'1 stop':1,'2 stops':2,'3 stops':3,'4 stops':4,np.nan:1})
print('df[`Total_Stops_Optimized`] = ',df['Total_Stops_Optimized'])
df.head(2)

#5) Total_Stops_Optimized should have zero null values now, as np.nan is replaced with 1 from the above map
print(df[df['Total_Stops_Optimized'].isnull()])

Unique values in `Total Stops`  [nan]
Total stops is null in the following rows =             Airline Date_of_Journey    Source Destination  \
0           IndiGo      24/03/2019  Banglore   New Delhi   
1        Air India       1/05/2019   Kolkata    Banglore   
2      Jet Airways       9/06/2019     Delhi      Cochin   
3           IndiGo      12/05/2019   Kolkata    Banglore   
4           IndiGo      01/03/2019  Banglore   New Delhi   
...            ...             ...       ...         ...   
10678     Air Asia       9/04/2019   Kolkata    Banglore   
10679    Air India      27/04/2019   Kolkata    Banglore   
10680  Jet Airways      27/04/2019  Banglore       Delhi   
10681      Vistara      01/03/2019  Banglore   New Delhi   
10682    Air India       9/05/2019     Delhi      Cochin   

                       Route  Arrival_Time Duration  Total_Stops  \
0                  BLR → DEL  01:10 22 Mar   2h 50m          NaN   
1      CCU → IXR → BBI → BLR         13:15   7h 25m         

In [ ]:
# Optimizing 'Route' column  (e.g. DFW to BLG)
# 1) Deleting 'Route' column as that has no effect on the total price
df.drop('Route',axis=1,inplace=True)
df.head(2)

,Airline,Date_of_Journey,Source,Destination,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Total_Stops2,Departure_hour,Departure_min,Total_Stops_Optimized
0,IndiGo,24/03/2019,Banglore,New Delhi,01:10 22 Mar,2h 50m,NaN,No info,3897,1,22,20,1
1,Air India,1/05/2019,Kolkata,Banglore,13:15,7h 25m,NaN,No info,7662,1,5,50,1


In [ ]:
# Optimizing 'Duration' column  (e.g. 2h 50m to get total minutes)
# Extract hours and minutes into new columns
# Nan values are replaced with '0'
df['Duration_Hours'] = (
    df['Duration']
    .str.extract(r'(\d+)\s*h')
    .astype(float)
    .fillna(0)
    .astype(int)
)

df['Duration_Minutes'] = (
    df['Duration']
    .str.extract(r'(\d+)\s*m')
    .astype(float)
    .fillna(0)
    .astype(int)
)

#Get total duration in minutes (hours * 60 + minutes)
df['Total_Duration'] = df['Duration_Hours'] * 60 + df['Duration_Minutes']
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Duration_Hours,Duration_Minutes,Total_Duration
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,2,50,170
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,7,25,445
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,19,0,1140
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,5,25,325
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,4,45,285


In [ ]:
# Optimizing of Category based data - Replacing with One Hot Encoding (OHE)
# Category based columns = Airline, Source, Additional_info
#1) Print all unique values across these columns
#2) Apply One Hot Encoding on these columns - Fit and Transform - Generate a Sparse Matrix
#3) Create new columns which coverted Classification data into Numerical data  - Convert Spare Matrix to a Dataframe

print ('Airline classification data: ',df['Airline'].unique());
print ('Source classification data: ',df['Source'].unique());
print ('Additional_Info classification data: ',df['Additional_Info'].unique());

#2) Apply One Hot Encoding on these columns - Fit and Transform - Generate a Sparse Matrix
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder()
encoder.fit_transform(df[['Airline','Source','Destination']]).toarray()

#3) Create new columns which coverted Classification data into Numerical data - Convert Spare Matrix to a Dataframe
# Put all the results in a dataframe
pd.DataFrame(encoder.fit_transform(df[['Airline','Source','Destination']]).toarray(),columns=encoder.get_feature_names_out())



Airline classification data:  ['IndiGo' 'Air India' 'Jet Airways' 'SpiceJet' 'Multiple carriers' 'GoAir'
 'Vistara' 'Air Asia' 'Vistara Premium economy' 'Jet Airways Business'
 'Multiple carriers Premium economy' 'Trujet']
Source classification data:  ['Banglore' 'Kolkata' 'Delhi' 'Chennai' 'Mumbai']
Additional_Info classification data:  ['No info' 'In-flight meal not included' 'No check-in baggage included'
 '1 Short layover' 'No Info' '1 Long layover' 'Change airports'
 'Business class' 'Red-eye flight' '2 Long layover']


,Airline_Air Asia,Airline_Air India,Airline_GoAir,Airline_IndiGo,Airline_Jet Airways,Airline_Jet Airways Business,Airline_Multiple carriers,Airline_Multiple carriers Premium economy,Airline_SpiceJet,Airline_Trujet,...,Source_Chennai,Source_Delhi,Source_Kolkata,Source_Mumbai,Destination_Banglore,Destination_Cochin,Destination_Delhi,Destination_Hyderabad,Destination_Kolkata,Destination_New Delhi
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10678,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10679,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
10680,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
10681,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
